# 📘 Sesión 4: Generadores e Iteradores

---

## 🎯 Objetivos

- Entender el protocolo de iteración
- Crear generadores con yield
- Usar expresiones generadoras
- Implementar pipelines de datos eficientes

## 1. Protocolo de Iteración

In [ ]:
# Iterador personalizado
class Contador:
    """Iterador que cuenta hasta un límite."""
    
    def __init__(self, limite):
        self.limite = limite
        self.actual = 0
    
    def __iter__(self):
        return self
    
    def __next__(self):
        if self.actual >= self.limite:
            raise StopIteration
        self.actual += 1
        return self.actual

for num in Contador(5):
    print(num, end=' ')

In [ ]:
# Clase iterable vs iterador
class Rango:
    """Clase iterable (puede iterarse múltiples veces)."""
    
    def __init__(self, inicio, fin):
        self.inicio = inicio
        self.fin = fin
    
    def __iter__(self):
        # Retorna un NUEVO iterador cada vez
        return RangoIterador(self.inicio, self.fin)

class RangoIterador:
    def __init__(self, inicio, fin):
        self.actual = inicio
        self.fin = fin
    
    def __iter__(self):
        return self
    
    def __next__(self):
        if self.actual >= self.fin:
            raise StopIteration
        valor = self.actual
        self.actual += 1
        return valor

mi_rango = Rango(1, 4)
print("Primera vez:", list(mi_rango))
print("Segunda vez:", list(mi_rango))  # Funciona de nuevo

## 2. Generadores con yield

In [ ]:
# Generador básico
def contador(limite):
    """Generador que cuenta hasta un límite."""
    n = 1
    while n <= limite:
        yield n
        n += 1

# Es más simple que la clase!
for num in contador(5):
    print(num, end=' ')

print("\n")

# El generador produce valores bajo demanda (lazy)
gen = contador(3)
print(f"Tipo: {type(gen)}")
print(f"Siguiente: {next(gen)}")
print(f"Siguiente: {next(gen)}")
print(f"Siguiente: {next(gen)}")

In [ ]:
# Generador infinito
def numeros_pares():
    """Genera números pares infinitamente."""
    n = 0
    while True:
        yield n
        n += 2

# Tomar solo lo que necesitamos
from itertools import islice

primeros_10_pares = list(islice(numeros_pares(), 10))
print(f"Primeros 10 pares: {primeros_10_pares}")

In [ ]:
# Generador de Fibonacci
def fibonacci():
    """Genera la secuencia de Fibonacci infinitamente."""
    a, b = 0, 1
    while True:
        yield a
        a, b = b, a + b

fib = fibonacci()
print("Fibonacci:", [next(fib) for _ in range(15)])

In [ ]:
# Leer archivos grandes eficientemente
def leer_lineas(contenido):
    """Simula leer un archivo línea por línea."""
    for linea in contenido.split('\n'):
        yield linea.strip()

texto = """Línea 1
Línea 2
Línea 3
Línea 4"""

for linea in leer_lineas(texto):
    print(f"Procesando: {linea}")

## 3. Expresiones Generadoras

In [ ]:
# Comparación de memoria
import sys

# Lista - almacena todos los elementos en memoria
lista = [x**2 for x in range(1000)]
print(f"Lista: {sys.getsizeof(lista)} bytes")

# Generador - almacena solo la lógica
generador = (x**2 for x in range(1000))
print(f"Generador: {sys.getsizeof(generador)} bytes")

In [ ]:
# Expresiones generadoras en funciones
numeros = [1, 2, 3, 4, 5]

# sum() acepta generadores directamente
suma_cuadrados = sum(x**2 for x in numeros)
print(f"Suma de cuadrados: {suma_cuadrados}")

# any() y all() con generadores
hay_pares = any(x % 2 == 0 for x in numeros)
todos_positivos = all(x > 0 for x in numeros)
print(f"Hay pares: {hay_pares}")
print(f"Todos positivos: {todos_positivos}")

## 4. yield from y Subgeneradores

In [ ]:
# yield from delega a otro generador
def generador_compuesto():
    yield from range(3)
    yield from 'ABC'
    yield from [10, 20, 30]

print(list(generador_compuesto()))

In [ ]:
# Aplanar lista con yield from
def aplanar(lista):
    """Aplana una lista anidada usando generadores."""
    for elemento in lista:
        if isinstance(elemento, list):
            yield from aplanar(elemento)
        else:
            yield elemento

anidada = [1, [2, 3], [4, [5, 6]], 7]
print(list(aplanar(anidada)))

## 5. Corrutinas con send()

In [ ]:
# Corrutina que recibe valores
def acumulador():
    """Corrutina que acumula valores."""
    total = 0
    while True:
        valor = yield total
        if valor is None:
            break
        total += valor

acc = acumulador()
next(acc)  # Inicializar (llegar al primer yield)

print(acc.send(10))  # Enviar 10, recibir total
print(acc.send(20))  # Enviar 20
print(acc.send(5))   # Enviar 5

In [ ]:
# Corrutina como pipeline
def filtrar_pares(destino):
    """Filtra pares y los envía al destino."""
    while True:
        valor = yield
        if valor % 2 == 0:
            destino.send(valor)

def imprimir():
    """Imprime los valores recibidos."""
    while True:
        valor = yield
        print(f"Recibido: {valor}")

# Crear pipeline
printer = imprimir()
next(printer)

filtro = filtrar_pares(printer)
next(filtro)

# Enviar datos
for i in range(10):
    filtro.send(i)

## 6. itertools - Herramientas de Iteración

In [ ]:
from itertools import (
    count, cycle, repeat,  # Infinitos
    chain, islice, takewhile, dropwhile,  # Manipulación
    product, permutations, combinations,  # Combinatorios
    groupby, accumulate  # Agrupación
)

# count - contador infinito
print("count:", list(islice(count(10, 2), 5)))  # 10, 12, 14...

# cycle - ciclo infinito
print("cycle:", list(islice(cycle('ABC'), 7)))

# chain - concatenar iterables
print("chain:", list(chain([1, 2], [3, 4], [5])))

# takewhile/dropwhile - tomar/descartar mientras condición
nums = [1, 3, 5, 7, 2, 4, 6]
print("takewhile <6:", list(takewhile(lambda x: x < 6, nums)))
print("dropwhile <6:", list(dropwhile(lambda x: x < 6, nums)))

In [ ]:
# Combinatorios
from itertools import product, permutations, combinations

# Producto cartesiano
print("Producto:", list(product('AB', [1, 2])))

# Permutaciones
print("Permutaciones:", list(permutations('ABC', 2)))

# Combinaciones
print("Combinaciones:", list(combinations('ABCD', 2)))

In [ ]:
# groupby - agrupar elementos consecutivos
from itertools import groupby

datos = [('A', 1), ('A', 2), ('B', 3), ('B', 4), ('A', 5)]

for clave, grupo in groupby(datos, key=lambda x: x[0]):
    print(f"{clave}: {list(grupo)}")

## 7. Pipelines de Datos

In [ ]:
# Pipeline de procesamiento de datos
def leer_datos():
    """Simula lectura de datos."""
    datos = [
        "  Juan,25,Madrid  ",
        "Ana,30,Barcelona",
        "  ,28,Valencia",  # Inválido
        "Carlos,35,Sevilla"
    ]
    for linea in datos:
        yield linea

def limpiar(lineas):
    """Limpia espacios."""
    for linea in lineas:
        yield linea.strip()

def parsear(lineas):
    """Convierte a diccionario."""
    for linea in lineas:
        partes = linea.split(',')
        if len(partes) == 3:
            yield {
                'nombre': partes[0],
                'edad': int(partes[1]),
                'ciudad': partes[2]
            }

def filtrar_validos(registros):
    """Filtra registros válidos."""
    for r in registros:
        if r['nombre']:  # Nombre no vacío
            yield r

def transformar(registros):
    """Transforma datos."""
    for r in registros:
        r['nombre'] = r['nombre'].upper()
        r['es_adulto'] = r['edad'] >= 18
        yield r

# Pipeline completo (lazy, eficiente en memoria)
pipeline = transformar(
    filtrar_validos(
        parsear(
            limpiar(
                leer_datos()
            )
        )
    )
)

for registro in pipeline:
    print(registro)

---
## 🏋️ Ejercicios Resueltos

In [ ]:
# Ejercicio 1: Generador de números primos
def primos():
    """Genera números primos infinitamente."""
    def es_primo(n):
        if n < 2:
            return False
        for i in range(2, int(n**0.5) + 1):
            if n % i == 0:
                return False
        return True
    
    n = 2
    while True:
        if es_primo(n):
            yield n
        n += 1

print("Primeros 10 primos:", list(islice(primos(), 10)))

In [ ]:
# Ejercicio 2: Ventana deslizante
def ventana_deslizante(iterable, n):
    """Genera ventanas de tamaño n."""
    from collections import deque
    ventana = deque(maxlen=n)
    
    for elemento in iterable:
        ventana.append(elemento)
        if len(ventana) == n:
            yield tuple(ventana)

datos = [1, 2, 3, 4, 5, 6]
print("Ventanas de 3:", list(ventana_deslizante(datos, 3)))

In [ ]:
# Ejercicio 3: Merge de iteradores ordenados
import heapq

def merge_ordenados(*iterables):
    """Combina múltiples iteradores ordenados."""
    return heapq.merge(*iterables)

a = [1, 4, 7]
b = [2, 5, 8]
c = [3, 6, 9]

print("Merged:", list(merge_ordenados(a, b, c)))

---
## 📝 Ejercicios para Practicar

In [ ]:
# Ejercicio 1: Generador de chunks
def chunks(iterable, n):
    """Divide en grupos de tamaño n."""
    pass

# list(chunks([1,2,3,4,5,6,7], 3)) -> [[1,2,3], [4,5,6], [7]]

In [ ]:
# Ejercicio 2: Generador que alterna entre iterables
def alternar(*iterables):
    """Alterna elementos de múltiples iterables."""
    pass

# list(alternar('AB', '12')) -> ['A', '1', 'B', '2']

In [ ]:
# Ejercicio 3: Pipeline para procesar logs
# Implementar: leer -> filtrar errores -> extraer timestamp -> contar por hora
logs = [
    "2024-01-15 10:30:00 INFO Started",
    "2024-01-15 10:31:00 ERROR Connection failed",
    "2024-01-15 11:00:00 ERROR Timeout",
    "2024-01-15 11:30:00 INFO Completed"
]

# Tu pipeline aquí

---
## 🎯 Resumen

- **Iteradores**: Objetos con `__iter__` y `__next__`
- **Generadores**: Funciones con `yield` (más simples)
- **Expresiones generadoras**: `(x for x in iterable)`
- **yield from**: Delega a subgeneradores
- **itertools**: Herramientas para iteración eficiente
- **Pipelines**: Composición de generadores